<a href="https://colab.research.google.com/github/KorvenDalas/w1834001_DataVisualisation_Y6/blob/main/Data_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# w1834001 Data Visualisation

import os

def list_files_in_folder(folder_path):
    # Initialize an empty list to store file names
    file_list = []

    # Iterate over all the files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a regular file (not a directory)
        if os.path.isfile(os.path.join(folder_path, file_name)):
            # Add the file name to the list
            file_list.append(file_name)

    return file_list

# Provide the path to the folder you want to list the files of
folder_path = '/content/Datasets_Boroughs'
files = list_files_in_folder(folder_path)
for file in files:
    print(file)

All London PM25.xlsm
All London NO2.xlsm


In [ ]:
import openpyxl

def add_particle_column(file_name):
    # Load the workbook
    workbook = openpyxl.load_workbook(file_name, keep_vba = True)

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Insert two new columns at the beginning
        sheet.insert_cols(1)
        sheet.insert_cols(1)

        # Set the header for the new columns
        sheet['A1'] = 'Borough'
        sheet['B1'] = 'Particle'

        # Fill the new columns
        if 'NO2' in file_name:
            borough, particle = file_name.split(" NO2")
            particle = "NO2"
            for row in range(2, sheet.max_row + 1):
                sheet[f'A{row}'] = borough
                sheet[f'B{row}'] = particle
        else:
            borough, particle = file_name.split(" PM25")
            particle = "PM25"
            for row in range(2, sheet.max_row + 1):
                sheet[f'A{row}'] = borough
                sheet[f'B{row}'] = particle

    # Save the workbook
    workbook.save(file_name)

os.chdir('/content/Datasets_Boroughs')

for file in files:
    add_particle_column(file)

In [ ]:
import pandas as pd

# Function to read all sheets from an Excel file into a dictionary
def read_excel_sheets(filename):
    with pd.ExcelFile(filename) as xls:
        return {sheet_name: xls.parse(sheet_name) for sheet_name in xls.sheet_names}

# Reading all sheets from all files into a dictionary of DataFrames
all_data = {}
for file in files:
    file_data = read_excel_sheets(file)
    for sheet_name, df in file_data.items():
        # Concatenate the DataFrames from the same sheet across all files
        if sheet_name in all_data:
            all_data[sheet_name] = pd.concat([all_data[sheet_name], df])
        else:
            all_data[sheet_name] = df

# Create a new Excel writer object
with pd.ExcelWriter('_London_all.xlsx', engine = 'openpyxl') as writer:
    # Write each combined DataFrame to a separate sheet in the new Excel file
    for sheet_name, df in all_data.items():
        df.to_excel(writer, sheet_name = sheet_name, index = False)

In [ ]:
# current_directory = os.getcwd()

# # List all files and directories in the current directory
# items_in_directory = os.listdir(current_directory)

# # Loop through the items and remove each one
# for item in items_in_directory:
#     item_path = os.path.join(current_directory, item)
#     if os.path.isfile(item_path):
#         os.remove(item_path)
#     elif os.path.isdir(item_path):
#         shutil.rmtree(item_path)